In [1]:
!pip install tensorflow==2.16.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: ml-dtypes
    F

In [1]:
!pip uninstall -y jax jaxlib flax optax orbax-checkpoint


Found existing installation: jax 0.7.2
Uninstalling jax-0.7.2:
  Successfully uninstalled jax-0.7.2
Found existing installation: jaxlib 0.7.2
Uninstalling jaxlib-0.7.2:
  Successfully uninstalled jaxlib-0.7.2
Found existing installation: flax 0.10.7
Uninstalling flax-0.10.7:
  Successfully uninstalled flax-0.10.7
Found existing installation: optax 0.2.6
Uninstalling optax-0.2.6:
  Successfully uninstalled optax-0.2.6
Found existing installation: orbax-checkpoint 0.11.31
Uninstalling orbax-checkpoint-0.11.31:
  Successfully uninstalled orbax-checkpoint-0.11.31


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!ls /content/drive/MyDrive/5TH_SEM_EL


cnn64.h5	 grams_encoder_part_4.h5  grams_fnn.h5
func_dll_fnn.h5  grams_fnn_beta_1.h5	  sequencer.h5


In [8]:
from tensorflow.keras.models import load_model

grams_fnn = load_model(
    '/content/drive/MyDrive/5TH_SEM_EL/grams_fnn.h5',
    compile=False
)

func_dll_fnn = load_model(
    '/content/drive/MyDrive/5TH_SEM_EL/func_dll_fnn.h5',
    compile=False
)



In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [10]:
TIME_STEPS = 128     # opcode sequence length
FEATURES = 49       # opcode vocabulary / embedding size

model = Sequential([
    Input(shape=(TIME_STEPS, FEATURES)),

    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),

    GRU(64, return_sequences=False),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [11]:
from tensorflow.keras.optimizers import Adam

grams_fnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)


In [12]:
func_dll_fnn.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


In [13]:
TIME_STEPS = 128
FEATURES = 49

sequencer = Sequential([
    Input(shape=(TIME_STEPS, FEATURES)),

    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),

    GRU(64),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [14]:
sequencer.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


In [15]:
from tensorflow.keras.optimizers import Adam

grams_fnn.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

func_dll_fnn.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

sequencer.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [17]:
import numpy as np

# Input size must match the model
X_grams = np.random.rand(1, grams_fnn.input_shape[1])
X_dll = np.random.rand(1, func_dll_fnn.input_shape[1])
X_seq = np.random.rand(1, 128, 49)


In [18]:
p_grams = grams_fnn.predict(X_grams)[0][0]
p_dll   = func_dll_fnn.predict(X_dll)[0][0]
p_seq   = sequencer.predict(X_seq)[0][0]

print("4-gram FNN:", p_grams)
print("DLL FNN:", p_dll)
print("Sequence:", p_seq)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step
4-gram FNN: 1.0
DLL FNN: 1.0
Sequence: 0.51860785


In [19]:
import requests
import numpy as np

CNN_API = " https://synthia-unantagonistic-lexie.ngrok-free.dev/predict"

X_img = np.random.rand(64, 64, 1)

response = requests.post(
    CNN_API,
    json={"image": X_img.tolist()}
)

p_cnn = response.json()["cnn_score"]
print("p_cnn:", p_cnn)


p_cnn: 0.7322263717651367


In [20]:
final_score = (p_cnn + p_grams + p_dll + p_seq) / 4

if final_score >= 0.5:
    print("FINAL RESULT: MALWARE")
else:
    print("FINAL RESULT: BENIGN")


FINAL RESULT: MALWARE
